# Lesson 2: Sequential Chats and Customer Onboarding

## Setup

In [ ]:
# import os
# from dotenv import load_dotenv, find_dotenv

# def load_env():
#     _ = load_dotenv(find_dotenv())

# def get_openai_api_key():
#     load_env()
#     openai_api_key = os.getenv("OPENAI_API_KEY")
#     return openai_api_key

# OPENAI_API_KEY = get_openai_api_key()
# llm_config = {"model": "gpt-3.5-turbo"}

In [1]:
llm_config = {
    "config_list": [
        {
            "model": "llama3.1",  # 
            "api_key": "ollama",  # 
            "base_url": "http://localhost:11434/v1",  
        }
    ],
    "cache_seed": None,  # Turns off caching, useful for testing different models
}

## Creating the needed agents

In [2]:
from autogen import ConversableAgent

c:\Users\OON YEE SEM\Documents\OON YEE SEM\Self Learning\Coursera Autogen\venv\lib\site-packages\flaml\__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


In [3]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. 
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [4]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [5]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [6]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [13]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "last_msg",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 1,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "last_msg",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "last_msg",
    },
]

## Start the onboarding process

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.

In [14]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding Personal Information Agent):

John and Malaysia

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Topic preference Agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
John and Malaysia

----------------------------------------------

## Print out the summary

In [15]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

John and Malaysia


stock market


What a great combination!

Hey John from Malaysia! Did you know that on the KLCI (Kuala Lumpur Composite Index), which is the main stock market index in Malaysia, November 24th is considered "Black Monday"? It's because on this day back in 1989, the index plummeted by 21.5% - the largest one-day decline ever!

But don't worry, John, we can discuss more positive things about the Malaysian economy! For instance, did you know that Malaysia has a thriving e-commerce sector? In fact, it's estimated that online shopping will contribute up to RM200 billion (that's about USD 50 billion) to the country's GDP by 2025!

Would you like to learn more fun facts or maybe jokes related to John and Malaysia? Or perhaps we can explore some interesting stories about the stock market? What's your pick, my friend?




## Print out the cost

In [16]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0, 'llama3.1': {'cost': 0, 'prompt_tokens': 622, 'completion_tokens': 54, 'total_tokens': 676}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'llama3.1': {'cost': 0, 'prompt_tokens': 622, 'completion_tokens': 54, 'total_tokens': 676}}}


{'usage_including_cached_inference': {'total_cost': 0, 'llama3.1': {'cost': 0, 'prompt_tokens': 385, 'completion_tokens': 4, 'total_tokens': 389}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'llama3.1': {'cost': 0, 'prompt_tokens': 385, 'completion_tokens': 4, 'total_tokens': 389}}}


{'usage_including_cached_inference': {'total_cost': 0, 'llama3.1': {'cost': 0, 'prompt_tokens': 1038, 'completion_tokens': 599, 'total_tokens': 1637}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'llama3.1': {'cost': 0, 'prompt_tokens': 1038, 'completion_tokens': 599, 'total_tokens': 1637}}}




In [18]:
import pprint
pprint.pprint(chat_results)

[ChatResult(chat_id=None,
            chat_history=[{'content': "Hello, I'm here to help you get started "
                                      'with our product.Could you tell me your '
                                      'name and location?',
                           'role': 'assistant'},
                          {'content': 'John and Malaysia', 'role': 'user'}],
            summary='John and Malaysia',
            cost={'usage_excluding_cached_inference': {'llama3.1': {'completion_tokens': 54,
                                                                    'cost': 0,
                                                                    'prompt_tokens': 622,
                                                                    'total_tokens': 676},
                                                       'total_cost': 0},
                  'usage_including_cached_inference': {'llama3.1': {'completion_tokens': 54,
                                                                   